In [27]:
import os

import pandas as pd
import numpy as np

os.chdir("../src/")
from utils.data_prep import data_prep as dp

In [9]:
X_train = pd.read_csv("../data/train_values.csv", index_col=0)
X_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [29]:
dp.cardinalidade(X_train)

,Atributo,Cardinalidade,Valores
18,has_superstructure_mud_mortar_brick,2,"[0, 1]"
20,has_superstructure_timber,2,"[0, 1]"
21,has_superstructure_bamboo,2,"[0, 1]"
22,has_superstructure_rc_non_engineered,2,"[0, 1]"
23,has_superstructure_rc_engineered,2,"[0, 1]"
24,has_superstructure_other,2,"[0, 1]"
27,has_secondary_use,2,"[0, 1]"
19,has_superstructure_cement_mortar_brick,2,"[0, 1]"
28,has_secondary_use_agriculture,2,"[0, 1]"
30,has_secondary_use_rental,2,"[0, 1]"


In [11]:
y_train = pd.read_csv("../data/train_labels.csv", index_col=0)
y_train.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [28]:
dp.cardinalidade(y_train)

,Atributo,Cardinalidade,Valores
0,damage_grade,3,"[1, 2, 3]"
